In [1]:
%gui qt

In [2]:
import segyio
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm, trange

In [3]:
segy_data = segyio.open('sample.sgy')
number_of_traces = segy_data.tracecount

In [4]:
headers = segyio.tracefield.keys

segy_df = pd.DataFrame(columns=list(headers.keys()))

total_number_of_traces = segy_data.tracecount

for header_name, byte in tqdm(headers.items()):
    segy_df[header_name] = segy_data.attributes(byte)[:]

100%|██████████| 91/91 [00:03<00:00, 27.80it/s]


In [5]:
segy_df.head()

,TRACE_SEQUENCE_LINE,TRACE_SEQUENCE_FILE,FieldRecord,TraceNumber,EnergySourcePoint,CDP,CDP_TRACE,TraceIdentificationCode,NSummedTraces,NStackedTraces,...,TraceIdentifier,ScalarTraceHeader,SourceType,SourceEnergyDirectionMantissa,SourceEnergyDirectionExponent,SourceMeasurementMantissa,SourceMeasurementExponent,SourceMeasurementUnit,UnassignedInt1,UnassignedInt2
0,1,1,1700,1937,1700,1300,1300,1,44,0,...,0,0,0,0,0,0,0,0,0,0
1,2,2,1700,1937,1700,1301,1301,1,42,0,...,0,0,0,0,0,0,0,0,0,0
2,3,3,1700,1993,1700,1302,1302,1,42,0,...,0,0,0,0,0,0,0,0,0,0
3,4,4,1700,1994,1700,1303,1303,1,43,0,...,0,0,0,0,0,0,0,0,0,0
4,5,5,1700,1995,1700,1304,1304,1,45,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
trace_length = segy_df.loc[0, 'TRACE_SAMPLE_COUNT']
inline_no = segy_df['INLINE_3D']
xline_no = segy_df['CROSSLINE_3D']

segy_df['i'] = inline_no-inline_no.min()
segy_df['j'] = xline_no-xline_no.min()

volume = np.zeros([segy_df['i'].max()+1, segy_df['j'].max()+1, trace_length])

for trace_number in trange(0, number_of_traces):
    volume[segy_df.loc[trace_number, 'i']][segy_df.loc[trace_number, 'j']][:] = segy_data.trace[trace_number][:]
    

volume[volume>1e+8]=np.nan

100%|██████████| 10201/10201 [00:00<00:00, 22350.02it/s]


In [8]:
from mayavi import mlab

mlab.show_pipeline()

source = mlab.pipeline.scalar_field(volume)
source.spacing = [1, 1, -1]

for axis in ['x', 'z']:
    plane = mlab.pipeline.image_plane_widget(source, 
                                    plane_orientation='{}_axes'.format(axis),
                                    slice_index=100, colormap='gray')
mlab.outline()
mlab.show()

Exception occurred in traits notification handler for object: <mayavi.core.engine.Engine object at 0x135e63d58>, trait: children_ui_list, old value: scenes_items, new value: [<mayavi.core.scene.Scene object at 0x140749b48>]
Traceback (most recent call last):
  File "/Users/tamiminaser/anaconda/lib/python3.6/site-packages/traits/trait_notifiers.py", line 591, in _dispatch_change_event
    self.dispatch(handler, *args)
  File "/Users/tamiminaser/anaconda/lib/python3.6/site-packages/traits/trait_notifiers.py", line 695, in dispatch
    handler(*args)
  File "/Users/tamiminaser/anaconda/lib/python3.6/site-packages/traitsui/qt4/tree_editor.py", line 1524, in _children_replaced
    self._delete_node(cnid)
  File "/Users/tamiminaser/anaconda/lib/python3.6/site-packages/traitsui/qt4/tree_editor.py", line 457, in _delete_node
    self._delete_node(cnid)
  File "/Users/tamiminaser/anaconda/lib/python3.6/site-packages/traitsui/qt4/tree_editor.py", line 457, in _delete_node
    self._delete_node(c

AttributeError: 'ArraySource' object has no attribute '_listener_cache'

AttributeError: 'ModuleManager' object has no attribute '_listener_cache'

AttributeError: 'ImagePlaneWidget' object has no attribute '_listener_cache'